In [6]:
from logging import basicConfig
from threading import Thread
from time import sleep
from kazoo.client import KazooClient
import random
import sys

In [7]:
ACTION_COMMIT =   b'commit'
ACTION_ROLLBACK = b'rollback'
WAIT_TIME = 5

basicConfig()

In [8]:
class Client(Thread):
  def __init__(self, root: str, id: int):
    super().__init__()
    self.url = f"{root}/{id}"
    self.root = root
    self.id = id

  def run(self):
        client = KazooClient()
        client.start()

        value = ACTION_COMMIT if random.random() > 0.5 else ACTION_ROLLBACK
        print(f"Client {self.id} request {value.decode()}")
        
        client.create(self.url, value, ephemeral=True)
        
        @client.DataWatch(self.url)
        def watch_self(data, stat):
            if stat.version > 0:
                print(f"Client {self.id} do {data.decode()}")

        sleep(WAIT_TIME)
        client.stop()
        client.close()

In [9]:
class Coordinator():
    def main(self):
        coord = KazooClient()
        coord.start()

        if coord.exists("/coord"):
            coord.delete("/coord", recursive=True)

        coord.create("/coord")
        coord.create("/coord/transaction")

        n_clients = 5

        def solution():
            clients = coord.get_children("/coord/transaction")
            commits, rollbacks = 0, 0
            
            for clt in clients:
                commits += int(coord.get(f"/coord/transaction/{clt}")[0] == ACTION_COMMIT)
                rollbacks += int(coord.get(f"/coord/transaction/{clt}")[0] == ACTION_ROLLBACK)

            for clt in clients:
                coord.set(f"/coord/transaction/{clt}", b"commit" if commits > rollbacks else ACTION_ROLLBACK)

        @coord.ChildrenWatch("/coord/transaction")
        def check_clients(clients):
            if len(clients) < n_clients:
                print(f"Waiting others clients: {clients}")

            elif len(clients) == n_clients:
                print("Check clients")
                solution()

        for x in range(5):
            begin=Client("/coord/transaction", x)
            begin.start()

In [11]:
Coordinator().main()

Waiting others clients: []
Client 0 request commit
Client 2 request commit
Client 3 request rollback
Client 4 request commit
Client 1 request rollback
Check clients
Client 0 do commit
Client 1 do commit
Client 2 do commit
Client 3 do commit
Client 4 do commit
Waiting others clients: []
